## Handling Missing Values in Large-scale ML Pipelines:

**Task 1**: Impute with Mean or Median
- Step 1: Load a dataset with missing values (e.g., Boston Housing dataset).
- Step 2: Identify columns with missing values.
- Step 3: Impute missing values using the mean or median of the respective columns.

In [ ]:
# write your code from here

**Task 2**: Impute with the Most Frequent Value
- Step 1: Use the Titanic dataset and identify columns with missing values.
- Step 2: Impute categorical columns using the most frequent value.

In [ ]:
# write your code from here

**Task 3**: Advanced Imputation - k-Nearest Neighbors
- Step 1: Implement KNN imputation using the KNNImputer from sklearn.
- Step 2: Explore how KNN imputation improves data completion over simpler methods.

In [ ]:
# write your code from here

## Feature Scaling & Normalization Best Practices:

**Task 1**: Standardization
- Step 1: Standardize features using StandardScaler.
- Step 2: Observe how standardization affects data distribution.

In [ ]:
# write your code from here

**Task 2**: Min-Max Scaling

- Step 1: Scale features to lie between 0 and 1 using MinMaxScaler.
- Step 2: Compare with standardization.

In [ ]:
# write your code from here

**Task 3**: Robust Scaling
- Step 1: Scale features using RobustScaler, which is useful for data with outliers.
- Step 2: Assess changes in data scaling compared to other scaling methods.

In [ ]:
# write your code from here

## Feature Selection Techniques:
### Removing Highly Correlated Features:

**Task 1**: Correlation Matrix
- Step 1: Compute correlation matrix.
- Step 2: Remove highly correlated features (correlation > 0.9).

In [ ]:
# write your code from here

### Using Mutual Information & Variance Thresholds:

**Task 2**: Mutual Information
- Step 1: Compute mutual information between features and target.
- Step 2: Retain features with high mutual information scores.

In [ ]:
# write your code from here

**Task 3**: Variance Threshold
- Step 1: Implement VarianceThreshold to remove features with low variance.
- Step 2: Analyze impact on feature space.

In [ ]:
# write your code from here

In [2]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import VarianceThreshold, mutual_info_classif
from sklearn.model_selection import train_test_split

# Utility: Safe Data Loader
def load_boston_dataset():
    try:
        data = load_boston()
        df = pd.DataFrame(data.data, columns=data.feature_names)
        df['TARGET'] = data.target
        return df
    except Exception as e:
        print(f"Error loading Boston dataset: {e}")
        return pd.DataFrame()

# Utility: Load Titanic with error handling
def load_titanic_dataset():
    try:
        url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
        df = pd.read_csv(url)
        return df
    except Exception as e:
        print(f"Error loading Titanic dataset: {e}")
        return pd.DataFrame()

# Missing Value Imputation Functions
def impute_mean(df):
    try:
        imputer = SimpleImputer(strategy='mean')
        return pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    except Exception as e:
        print(f"Mean imputation failed: {e}")
        return df

def impute_most_frequent(df, categorical_columns):
    try:
        imputer = SimpleImputer(strategy='most_frequent')
        df[categorical_columns] = imputer.fit_transform(df[categorical_columns])
        return df
    except Exception as e:
        print(f"Mode imputation failed: {e}")
        return df

def impute_knn(df, n_neighbors=3):
    try:
        imputer = KNNImputer(n_neighbors=n_neighbors)
        return pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
    except Exception as e:
        print(f"KNN imputation failed: {e}")
        return df

# Scaling Functions
def scale_standard(df):
    return pd.DataFrame(StandardScaler().fit_transform(df), columns=df.columns)

def scale_minmax(df):
    return pd.DataFrame(MinMaxScaler().fit_transform(df), columns=df.columns)

def scale_robust(df):
    return pd.DataFrame(RobustScaler().fit_transform(df), columns=df.columns)

# Feature Selection
def remove_highly_correlated_features(df, threshold=0.9):
    corr_matrix = df.corr().abs()
    upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > threshold)]
    return df.drop(columns=to_drop)

def select_by_mutual_info(df, target, threshold='median'):
    y = (target > np.median(target)).astype(int) if threshold == 'median' else target
    scores = mutual_info_classif(df, y)
    selected_cols = df.columns[scores > np.median(scores)]
    return df[selected_cols]

def apply_variance_threshold(df, threshold=0.01):
    selector = VarianceThreshold(threshold)
    return pd.DataFrame(selector.fit_transform(df), columns=df.columns[selector.get_support(indices=True)])

# ---------- MAIN EXECUTION ---------- #

# Load datasets
boston_df = load_boston_dataset()
titanic_df = load_titanic_dataset()

# Inject missing values
if not boston_df.empty:
    boston_df.iloc[0, 0] = np.nan

# Task 1: Mean Imputation
boston_imputed = impute_mean(boston_df.drop('TARGET', axis=1))

# Task 2: Most Frequent Imputation
titanic_imputed = impute_most_frequent(titanic_df.copy(), titanic_df.select_dtypes(include='object').columns)

# Task 3: KNN Imputation
boston_knn_imputed = impute_knn(boston_imputed)

# Scaling Tasks
boston_std = scale_standard(boston_imputed)
boston_minmax = scale_minmax(boston_imputed)
boston_robust = scale_robust(boston_imputed)

# Feature Selection Tasks
boston_uncorrelated = remove_highly_correlated_features(boston_imputed)
boston_mi_selected = select_by_mutual_info(boston_imputed, boston_df['TARGET'])
boston_variance = apply_variance_threshold(boston_imputed)

# Basic Verification/Unit Checks
assert not boston_std.isnull().values.any(), "StandardScaler failed: NaNs present"
assert not boston_minmax.isnull().values.any(), "MinMaxScaler failed: NaNs present"
assert not boston_robust.isnull().values.any(), "RobustScaler failed: NaNs present"

print("✅ All processing completed successfully. Data samples:")
print("\nMean Imputed Boston Data:\n", boston_imputed.head())
print("\nMost Frequent Imputed Titanic Data:\n", titanic_imputed.select_dtypes(include='object').head())
print("\nKNN Imputed Boston Data:\n", boston_knn_imputed.head())
print("\nStandard Scaled Data:\n", boston_std.head())
print("\nAfter Removing Correlated Features:\n", boston_uncorrelated.head())
print("\nMutual Info Selected Features:\n", boston_mi_selected.head())
print("\nVariance Thresholded Features:\n", boston_variance.head())


ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: